In [105]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import chi2_contingency
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
from sklearn.cluster import DBSCAN
from hdbscan import HDBSCAN, approximate_predict
from hdbscan.prediction import membership_vector
from sklearn.neighbors import NearestNeighbors
from collections import Counter
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from skopt import BayesSearchCV
from skopt.space import Integer, Categorical
from category_encoders import CatBoostEncoder
from copy import deepcopy
from skopt import BayesSearchCV
from skopt.space import Integer, Categorical

pd.set_option('display.max_columns', None)

In [106]:
flight_data_train = pd.read_csv('./flight_data_train_ts.csv')
flight_data_test = pd.read_csv('./flight_data_test_ts.csv')

In [107]:
flight_data_train['scheduledoffblocktime'] = pd.to_datetime(flight_data_train['scheduledoffblocktime'])
flight_data_test['scheduledoffblocktime'] = pd.to_datetime(flight_data_test['scheduledoffblocktime'])

# flight_data_train.sort_values(by='scheduledoffblocktime', inplace=True)
# flight_data_test.sort_values(by='scheduledoffblocktime', inplace=True)

Feature Selection

In [108]:
def chi_2(df, x, y):
    # Create a contingency table
    contingency_table = pd.crosstab(df[x], df[y])

    # Perform the Chi-Square test
    chi2, p, dof, expected = chi2_contingency(contingency_table)
    print(f"Chi-Square Statistic for {x} and {y}: {chi2}, p-value: {p}, dof:{dof}")

In [109]:
def cramers_v(df, x, y):
    # Create a contingency table
    contingency_table = pd.crosstab(df[x], df[y])
    
    # Perform the Chi-Square test
    chi2, p, _, _ = chi2_contingency(contingency_table)
    
    # Calculate Cramer's V
    n = contingency_table.sum().sum()
    min_dim = min(contingency_table.shape) - 1
    cor = np.sqrt(chi2 / (n * min_dim))
    print(f"Cramer's V  for {x} and {y}: {cor}, p-value: {p}")

In [110]:
flight_data_train.columns

Index(['scheduledoffblocktime', 'traffictypecode', 'airlinecode_iata',
       'destination_iata', 'aircraft_iata', 'isconnecting', 'publicgatenumber',
       'aircraftterminal', 'tmpf', 'dwpf', 'relh', 'drct', 'sknt', 'alti',
       'vsby', 'gust', 'skyc1', 'skyc2', 'skyl1', 'skyl2', 'finalflightstatus',
       'delay_in_secs'],
      dtype='object')

In [111]:
chi_2(flight_data_train, 'publicgatenumber', 'finalflightstatus')
cramers_v(flight_data_train, 'publicgatenumber', 'finalflightstatus')

Chi-Square Statistic for publicgatenumber and finalflightstatus: 3653.5185592382445, p-value: 0.0, dof:138
Cramer's V  for publicgatenumber and finalflightstatus: 0.14586302601258685, p-value: 0.0


In [112]:
chi_2(flight_data_train, 'destination_iata', 'finalflightstatus')
cramers_v(flight_data_train, 'destination_iata', 'finalflightstatus')

Chi-Square Statistic for destination_iata and finalflightstatus: 8301.845466494837, p-value: 0.0, dof:311
Cramer's V  for destination_iata and finalflightstatus: 0.21987552153760798, p-value: 0.0


In [113]:
chi_2(flight_data_train, 'aircraft_iata', 'finalflightstatus')
cramers_v(flight_data_train, 'aircraft_iata', 'finalflightstatus')

Chi-Square Statistic for aircraft_iata and finalflightstatus: 3711.169825246382, p-value: 0.0, dof:60
Cramer's V  for aircraft_iata and finalflightstatus: 0.14700935572653068, p-value: 0.0


In [114]:
chi_2(flight_data_train, 'airlinecode_iata', 'finalflightstatus')
cramers_v(flight_data_train, 'airlinecode_iata', 'finalflightstatus')

Chi-Square Statistic for airlinecode_iata and finalflightstatus: 9039.246389847638, p-value: 0.0, dof:149
Cramer's V  for airlinecode_iata and finalflightstatus: 0.22943288824568359, p-value: 0.0


In [115]:
# columns_to_drop = ['publicgatenumber']

# flight_data_train.drop(columns=columns_to_drop, axis=1, inplace=True)
# flight_data_test.drop(columns=columns_to_drop, axis=1, inplace=True)

In [116]:
departdatetime = flight_data_train['scheduledoffblocktime'].dt

flight_data_train['depart_day'] = departdatetime.day
flight_data_train['depart_month'] = departdatetime.month
flight_data_train['depart_dayofweek'] = departdatetime.dayofweek
flight_data_train['depart_minute'] = departdatetime.hour * 60 + departdatetime.minute
# Test
departdatetime = flight_data_test['scheduledoffblocktime'].dt
flight_data_test['depart_day'] = departdatetime.day
flight_data_test['depart_month'] = departdatetime.month
flight_data_test['depart_dayofweek'] = departdatetime.dayofweek
flight_data_test['depart_minute'] = departdatetime.hour * 60 + departdatetime.minute

flight_data_train.drop(columns=['scheduledoffblocktime'], axis=1, inplace=True)
flight_data_test.drop(columns=['scheduledoffblocktime'], axis=1, inplace=True)

In [117]:
X_train = flight_data_train.drop(columns=['delay_in_secs', 'finalflightstatus'], axis=1)
X_test = flight_data_test.drop(columns=['delay_in_secs', 'finalflightstatus'], axis=1)

y_train = flight_data_train['finalflightstatus']
y_test = flight_data_test['finalflightstatus']

In [118]:
y_train = y_train.map({'On-Time': 0, 'Delayed':1})
y_test = y_test.map({'On-Time': 0, 'Delayed':1})

In [119]:
X_train

,traffictypecode,airlinecode_iata,destination_iata,aircraft_iata,isconnecting,publicgatenumber,aircraftterminal,tmpf,dwpf,relh,drct,sknt,alti,vsby,gust,skyc1,skyc2,skyl1,skyl2,depart_day,depart_month,depart_dayofweek,depart_minute
0,PS,RA,KTM,332,0,F8,2,69.8,62.6,77.90,90.0,5.0,29.88,6.21,0,CAVOK,CAVOK,0.0,0.0,8,3,6,265
1,PS,ET,ADD,77W,0,D6,1,69.8,62.6,77.90,90.0,5.0,29.88,6.21,0,CAVOK,CAVOK,0.0,0.0,8,3,6,265
2,PS,6E,BOM,320,0,D7,1,69.8,62.6,77.90,90.0,5.0,29.88,6.21,0,CAVOK,CAVOK,0.0,0.0,8,3,6,270
3,PS,EK,MLE,77W,0,C14,3,68.0,60.8,77.75,110.0,6.0,29.88,5.59,0,NSC,CAVOK,0.0,0.0,8,3,6,275
4,PS,EK,CGK,77W,0,A8,3,68.0,60.8,77.75,110.0,6.0,29.88,5.59,0,NSC,CAVOK,0.0,0.0,8,3,6,275
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171715,PS,EK,BRU,77W,0,B29,3,95.0,66.2,39.00,350.0,12.0,29.71,4.35,0,NSC,CAVOK,0.0,0.0,6,5,4,860
171716,PS,EK,IST,77W,0,B10,3,95.0,66.2,39.00,350.0,12.0,29.71,4.35,0,NSC,CAVOK,0.0,0.0,6,5,4,860
171717,PS,EK,DUS,388,0,B17,3,95.0,66.2,39.00,350.0,12.0,29.71,4.35,0,NSC,CAVOK,0.0,0.0,6,5,4,860
171718,PS,EK,LIS,77W,0,B22,3,95.0,66.2,39.00,350.0,12.0,29.71,4.35,0,NSC,CAVOK,0.0,0.0,6,5,4,865


In [120]:
# from imblearn.over_sampling import SMOTENC

# smote = SMOTENC(random_state=42, categorical_features=[0, 1, 2, 3, 5, 16, 17])
# print('Original dataset shape %s' % Counter(y_train_cls))
# X_train, y_train_cls = smote.fit_resample(X_train, y_train_cls)
# print('Resampled dataset shape %s' % Counter(y_train_cls))


In [121]:
import pandas as pd
from category_encoders import CatBoostEncoder

high_cardinality_cols = ['airlinecode_iata', 'destination_iata', 'aircraft_iata', 'publicgatenumber']

catboost_encoder = CatBoostEncoder(cols=high_cardinality_cols, return_df=True)

X_train_encoded = catboost_encoder.fit_transform(X_train, y_train)

X_test_encoded = catboost_encoder.transform(X_test)

X_train = X_train_encoded
X_test = X_test_encoded

In [122]:
# one_hot_column =  ['skyc1', 'skyc2', 'traffictypecode', 'aircraftterminal', 'airlinecode_iata', 'destination_iata']
one_hot_column =  ['skyc1', 'skyc2', 'traffictypecode', 'aircraftterminal', ]

ohe = OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore')

encoded = ohe.fit_transform(X_train[one_hot_column])
ohe_new_columns = ohe.get_feature_names_out(one_hot_column)
encoded_df = pd.DataFrame(encoded, columns=ohe_new_columns)
X_train = pd.concat([X_train.drop(columns=one_hot_column), encoded_df], axis=1)

encoded = ohe.transform(X_test[one_hot_column])
encoded_df = pd.DataFrame(encoded, columns=ohe_new_columns)
X_test = pd.concat([X_test.drop(columns=one_hot_column), encoded_df], axis=1)



In [123]:
numerical_cols = ['tmpf', 'dwpf', 'relh', 'drct', 'sknt', 'alti', 'vsby', 'skyl1', 'skyl2']

In [124]:
scaler = MinMaxScaler(feature_range=(0, 1))
X_train[numerical_cols] = scaler.fit_transform(X_train[numerical_cols])
X_test[numerical_cols] = scaler.transform(X_test[numerical_cols])

In [125]:
# Create cyclic features for month and day
X_train['depart_month_sin'] = np.sin(2 * np.pi * X_train['depart_month'] / 12)
X_train['depart_month_cos'] = np.cos(2 * np.pi * X_train['depart_month'] / 12)
X_train['depart_day_sin'] = np.sin(2 * np.pi * X_train['depart_day'] / 31) 
X_train['depart_day_cos'] = np.cos(2 * np.pi * X_train['depart_day'] / 31)
X_train['depart_dayofweek_sin'] = np.sin(2 * np.pi * X_train['depart_dayofweek'] / 7) 
X_train['depart_dayofweek_cos'] = np.cos(2 * np.pi * X_train['depart_dayofweek'] / 7)
X_train['depart_minute_sin'] = np.sin(2 * np.pi * X_train['depart_minute'] / 1440) 
X_train['depart_minute_cos'] = np.cos(2 * np.pi * X_train['depart_minute'] / 1440)

X_test['depart_month_sin'] = np.sin(2 * np.pi * X_test['depart_month'] / 12)
X_test['depart_month_cos'] = np.cos(2 * np.pi * X_test['depart_month'] / 12)
X_test['depart_day_sin'] = np.sin(2 * np.pi * X_test['depart_day'] / 31)
X_test['depart_day_cos'] = np.cos(2 * np.pi * X_test['depart_day'] / 31)
X_test['depart_dayofweek_sin'] = np.sin(2 * np.pi * X_test['depart_dayofweek'] / 7) 
X_test['depart_dayofweek_cos'] = np.cos(2 * np.pi * X_test['depart_dayofweek'] / 7)
X_test['depart_minute_sin'] = np.sin(2 * np.pi * X_test['depart_minute'] / 1440)
X_test['depart_minute_cos'] = np.cos(2 * np.pi * X_test['depart_minute'] / 1440)

# Drop original features
X_train = X_train.drop(['depart_month', 'depart_day', 'depart_minute', 'depart_dayofweek'], axis=1)
X_test = X_test.drop(['depart_month', 'depart_day', 'depart_minute', 'depart_dayofweek'], axis=1)


In [126]:
X_train

,airlinecode_iata,destination_iata,aircraft_iata,isconnecting,publicgatenumber,tmpf,dwpf,relh,drct,sknt,alti,vsby,gust,skyl1,skyl2,skyc1_CAVOK,skyc1_FEW,skyc1_NSC,skyc1_OVC,skyc1_SCT,skyc1_VV,skyc2_CAVOK,skyc2_FEW,skyc2_NSC,skyc2_SCT,traffictypecode_PE,traffictypecode_PS,aircraftterminal_1,aircraftterminal_2,aircraftterminal_3,depart_month_sin,depart_month_cos,depart_day_sin,depart_day_cos,depart_dayofweek_sin,depart_dayofweek_cos,depart_minute_sin,depart_minute_cos
0,0.274493,0.274493,0.274493,0,0.274493,0.242424,0.650,0.767540,0.250000,0.185185,0.652632,1.000000,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,6.123234e-17,0.998717,-0.050649,-0.781831,0.623490,0.915311,0.402747
1,0.274493,0.274493,0.274493,0,0.274493,0.242424,0.650,0.767540,0.250000,0.185185,0.652632,1.000000,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,6.123234e-17,0.998717,-0.050649,-0.781831,0.623490,0.915311,0.402747
2,0.274493,0.274493,0.274493,0,0.274493,0.242424,0.650,0.767540,0.250000,0.185185,0.652632,1.000000,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,6.123234e-17,0.998717,-0.050649,-0.781831,0.623490,0.923880,0.382683
3,0.274493,0.274493,0.137247,0,0.274493,0.212121,0.625,0.765962,0.305556,0.222222,0.652632,0.899676,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,6.123234e-17,0.998717,-0.050649,-0.781831,0.623490,0.932008,0.362438
4,0.137247,0.274493,0.091498,0,0.274493,0.212121,0.625,0.765962,0.305556,0.222222,0.652632,0.899676,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,6.123234e-17,0.998717,-0.050649,-0.781831,0.623490,0.932008,0.362438
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171715,0.224980,0.155440,0.208386,0,0.225553,0.666667,0.700,0.358368,0.972222,0.444444,0.473684,0.699029,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.5,-8.660254e-01,0.937752,0.347305,-0.433884,-0.900969,-0.573576,-0.819152
171716,0.224991,0.239724,0.208400,0,0.175686,0.666667,0.700,0.358368,0.972222,0.444444,0.473684,0.699029,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.5,-8.660254e-01,0.937752,0.347305,-0.433884,-0.900969,-0.573576,-0.819152
171717,0.224988,0.194093,0.343861,0,0.262227,0.666667,0.700,0.358368,0.972222,0.444444,0.473684,0.699029,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.5,-8.660254e-01,0.937752,0.347305,-0.433884,-0.900969,-0.573576,-0.819152
171718,0.224985,0.285690,0.208397,0,0.187338,0.666667,0.700,0.358368,0.972222,0.444444,0.473684,0.699029,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.5,-8.660254e-01,0.937752,0.347305,-0.433884,-0.900969,-0.591310,-0.806445


In [127]:
# neighbors = NearestNeighbors(n_neighbors=39*2)
# neighbors_fit = neighbors.fit(X_train)
# distances, indices = neighbors_fit.kneighbors(X_train)
# avg_distance = distances.mean(axis=1)
# plt.plot(np.sort(avg_distance))
# plt.show()

In [128]:
# sc = DBSCAN(eps=1.4, min_samples=37*2, algorithm='kd_tree')
# clusters = pd.DataFrame(sc.fit_predict(X_train), columns=['Cluster'])
# clusters['Cluster'].value_counts()

: 

In [129]:
param_space = {
    'n_estimators': Integer(50, 200),
    'max_depth': Integer(5, 50),
    'min_samples_split': Integer(2, 10),
    'min_samples_leaf': Integer(1, 5),
    'max_features': Categorical(['sqrt', 'log2']),
    'criterion': Categorical(['gini', 'entropy', 'log_loss'])
}

# Apply SMOTE to balance the training data
smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

# Print class distribution before and after SMOTE
print("Original class distribution:")
print(pd.Series(y_train).value_counts())
print("\nBalanced class distribution:")
print(pd.Series(y_train_balanced).value_counts())

bayes_cv = BayesSearchCV(
    estimator=RandomForestClassifier(),
    search_spaces=param_space,
    n_iter=50,
    cv=3,
    n_jobs=-1,
    scoring='f1',
    random_state=42
)
bayes_cv.fit(X_train_balanced, y_train_balanced)
best_rf = bayes_cv.best_estimator_

y_pred = best_rf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Performance without clustering:")
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print("==========================================")
print()


Original class distribution:
finalflightstatus
0    124584
1     47136
Name: count, dtype: int64

Balanced class distribution:
finalflightstatus
0    124584
1    124584
Name: count, dtype: int64


/home/abdussaboor/.virtualenvs/mlr570-project/lib/python3.12/site-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,
/home/abdussaboor/.virtualenvs/mlr570-project/lib/python3.12/site-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


In [86]:
hdbscan_model = HDBSCAN(
    min_cluster_size=3000,      # Increase to avoid microclusters
    min_samples=5,              # Lower to reduce noise points
    cluster_selection_epsilon=0.5,  # Increase to reduce noise points
    cluster_selection_method='eom',  # 'eom' tends to produce more balanced clusters
    prediction_data=True,
    core_dist_n_jobs=-1
)
clusters_train = pd.DataFrame(hdbscan_model.fit_predict(X_train), columns=['Cluster'])
clusters_train['Cluster'].value_counts()

Cluster
 9    65112
 7    26393
 8    21306
 4    16190
-1    12950
 5     7957
 3     5910
 1     4793
 2     4486
 6     3515
 0     3108
Name: count, dtype: int64

In [87]:
clusters_test, _ = approximate_predict(hdbscan_model, X_test)
pd.Series(clusters_test).value_counts()

-1    6402
 9    6066
 8    2232
 4    1925
 7    1388
 1     612
 0     242
 2     181
 3      32
 6       1
Name: count, dtype: int64

In [88]:
probs_test = membership_vector(hdbscan_model, X_test.to_numpy())
pd.DataFrame(probs_test).head()


,0,1,2,3,4,5,6,7,8,9
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [89]:
X_train['cluster'] = clusters_train['Cluster']
X_test['cluster'] = clusters_test


In [96]:
rf_models = {}
metrics = {}
model_weights_f1 = {}
model_weights_cluster = {}
entropy_weights = {}

for cluster in np.unique(clusters_train):
    if cluster == -1:  # Ignore noise points (label -1)
        continue
    
    print(f'Cluster {cluster}')

    # Subset the training and test data for the cluster
    # Get cluster data
    X_cluster = X_train[X_train['cluster'] == cluster].drop(columns=['cluster'])
    y_cluster = y_train.loc[X_cluster.index]
    
    X_test_cluster = X_test[X_test['cluster'] == cluster].drop(columns=['cluster'])
    y_test_cluster = y_test.loc[X_test_cluster.index]
    
    # Apply SMOTE to balance the training data
    smote = SMOTE(random_state=42)
    X_train_balanced, y_train_balanced = smote.fit_resample(X_cluster, y_cluster)

# Print class distribution before and after SMOTE
    print("Original class distribution:")
    print(pd.Series(y_cluster).value_counts())
    print("\nBalanced class distribution:")
    print(pd.Series(y_train_balanced).value_counts())

    # Define objective function for hyperopt
    param_space = {
        'n_estimators': Integer(50, 200),
        'max_depth': Integer(5, 50),
        'min_samples_split': Integer(2, 10),
        'min_samples_leaf': Integer(1, 5),
        'max_features': Categorical(['sqrt', 'log2']),
        'criterion': Categorical(['gini', 'entropy', 'log_loss']),
    }

    # Use Bayesian optimization for hyperparameter tuning
    bayes_cv = BayesSearchCV(
        estimator=RandomForestClassifier(),
        search_spaces=param_space,
        n_iter=50,
        cv=5,
        n_jobs=-1,
        scoring='f1',
        random_state=42
    )
    bayes_cv.fit(X_train_balanced, y_train_balanced)
    best_rf = bayes_cv.best_estimator_
    rf_models[cluster] = best_rf
    
    # Cluster weighted
    cluster_weight = len(y_cluster) / len(y_train)
    model_weights_cluster[cluster] = cluster_weight
    
    # f1 weighted
    model_weights_f1[cluster] = bayes_cv.best_score_
    
    # entropy weighted
    predictions = best_rf.predict(X_cluster)
    relative_errors = (predictions != y_cluster).astype(int)
    p_error = relative_errors.sum() / len(relative_errors)
    entropy = -p_error * np.log(p_error + 1e-9) - (1 - p_error) * np.log(1 - p_error + 1e-9)
    weight = 1 - entropy
    entropy_weights[cluster] = weight
    
    print(f"Cluster {cluster} best f1 score: {bayes_cv.best_score_}")


Cluster 0
Original class distribution:
finalflightstatus
0    2231
1     877
Name: count, dtype: int64

Balanced class distribution:
finalflightstatus
0    2231
1    2231
Name: count, dtype: int64
Cluster 0 best f1 score: 0.5926467719816508
Cluster 1
Original class distribution:
finalflightstatus
0    3489
1    1304
Name: count, dtype: int64

Balanced class distribution:
finalflightstatus
0    3489
1    3489
Name: count, dtype: int64
Cluster 1 best f1 score: 0.5636199627475283
Cluster 2
Original class distribution:
finalflightstatus
0    3095
1    1391
Name: count, dtype: int64

Balanced class distribution:
finalflightstatus
0    3095
1    3095
Name: count, dtype: int64
Cluster 2 best f1 score: 0.5942078131591209
Cluster 3
Original class distribution:
finalflightstatus
0    4054
1    1856
Name: count, dtype: int64

Balanced class distribution:
finalflightstatus
0    4054
1    4054
Name: count, dtype: int64
Cluster 3 best f1 score: 0.5268872939019653
Cluster 4
Original class distributio

In [97]:
param_space = {
    'n_estimators': Integer(50, 200),
    'max_depth': Integer(5, 50),
    'min_samples_split': Integer(2, 10),
    'min_samples_leaf': Integer(1, 5),
    'max_features': Categorical(['sqrt', 'log2']),
    'criterion': Categorical(['gini', 'entropy', 'log_loss']),
    }
bayes_cv = BayesSearchCV(
    estimator=RandomForestClassifier(),
    search_spaces=param_space,
    n_iter=50,
    cv=3,
    n_jobs=-1,
    scoring='f1_weighted',
    random_state=42
)
bayes_cv.fit(X_train[X_train['cluster'] != -1].drop(columns=['cluster']), X_train['cluster'][X_train['cluster'] != -1])
print(bayes_cv.best_score_)
pre_classifier = bayes_cv.best_estimator_
pre_classifier_probabilities = pre_classifier.predict_proba(X_test.drop(columns=['cluster']))

0.9863383175936203


In [98]:
cluster_centers = {c: X_train[X_train['cluster'] == c].drop(columns=['cluster']).mean() for c in rf_models.keys()}

In [99]:
total_entropy_weight = sum(entropy_weights.values())
for model_cluster in entropy_weights:
    entropy_weights[model_cluster] /= total_entropy_weight

In [100]:
model_weights_f1

{0: 0.5926467719816508,
 1: 0.5636199627475283,
 2: 0.5942078131591209,
 3: 0.5268872939019653,
 4: 0.5866064925114153,
 5: 0.5743078202437166,
 6: 0.542693055731273,
 7: 0.5507658954819468,
 8: 0.5311992401565029,
 9: 0.5804514517876889}

In [101]:
# Make final predictions on the test set using weighted average
final_predictions_f1_weighted = []
final_predictions_cluster_weighted = []
final_predictions_probability_weighted = []
final_predictions_non_weighted = []
final_predictions_pre_classifier = []
final_predictions_entropy_weighted = []
all_y_true = []
for idx in X_test.index:
    cluster = X_test.loc[idx, 'cluster']
    if cluster == -1:
        # Find nearest cluster using euclidean distance to cluster centroids
        X_point = X_test.drop(columns=['cluster']).loc[idx]
        min_dist = float('inf')
        nearest_cluster = None
        
        for c in rf_models.keys():
            # Calculate distance to precalculated centroid
            centroid = cluster_centers[c]
            dist = np.sqrt(((X_point - centroid) ** 2).sum())
            if dist < min_dist:
                min_dist = dist
                nearest_cluster = c
                
        cluster = nearest_cluster
        
    probabilities_hdbscan = probs_test[idx]
    probabilities_pre_classifier = pre_classifier_probabilities[idx]
    
    votes_weighted_f1 = {}
    votes_weighted_cluster = {}
    votes_probability = {}
    votes_pre_classifier = {}
    votes_entropy_weighted = {}
    for model_cluster, model in rf_models.items():
        prediction = model.predict(X_test.drop(columns=['cluster']).loc[[idx]])[0]
        weight_f1 = model_weights_f1.get(model_cluster, 0)
        if prediction in votes_weighted_f1:
            votes_weighted_f1[prediction] += weight_f1
        else:
            votes_weighted_f1[prediction] = weight_f1

        weight_cluster = model_weights_cluster.get(model_cluster, 0)
        if prediction in votes_weighted_cluster:
            votes_weighted_cluster[prediction] += weight_cluster
        else:
            votes_weighted_cluster[prediction] = weight_cluster
            
        if prediction in votes_probability:
            votes_probability[prediction] += probabilities_hdbscan[model_cluster]
        else:
            votes_probability[prediction] = probabilities_hdbscan[model_cluster]
        
        if prediction in votes_pre_classifier:
            votes_pre_classifier[prediction] += probabilities_pre_classifier[model_cluster]
        else:
            votes_pre_classifier[prediction] = probabilities_pre_classifier[model_cluster]
            
        if prediction in votes_entropy_weighted:
            votes_entropy_weighted[prediction] += entropy_weights[model_cluster]
        else:
            votes_entropy_weighted[prediction] = entropy_weights[model_cluster]
        
        
    model = rf_models[cluster]
    prediction = model.predict(X_test.drop(columns=['cluster']).loc[[idx]])[0]
    final_predictions_non_weighted.append(round(prediction))   
    

    # Final prediction is the weighted average
    final_prediction_f1_weighted = max(votes_weighted_f1, key=votes_weighted_f1.get)
    final_predictions_f1_weighted.append(final_prediction_f1_weighted)
    
    final_prediction_cluster_weighted = max(votes_weighted_cluster, key=votes_weighted_cluster.get)
    final_predictions_cluster_weighted.append(final_prediction_cluster_weighted)
    
    final_prediction_probability_weighted = max(votes_probability, key=votes_probability.get)
    final_predictions_probability_weighted.append(final_prediction_probability_weighted)
    
    final_prediction_pre_classifier = max(votes_pre_classifier, key=votes_pre_classifier.get)
    final_predictions_pre_classifier.append(final_prediction_pre_classifier)
    
    final_prediction_entropy_weighted = max(votes_entropy_weighted, key=votes_entropy_weighted.get)
    final_predictions_entropy_weighted.append(final_prediction_entropy_weighted)
    
    all_y_true.append(y_test.loc[idx])


In [104]:
# Calculate overall metrics
##########################
# Weighted Average F1 Ensemble
##########################
overall_accuracy = accuracy_score(all_y_true, final_predictions_f1_weighted)
overall_precision = precision_score(all_y_true, final_predictions_f1_weighted)
overall_recall = recall_score(all_y_true, final_predictions_f1_weighted)
overall_f1 = f1_score(all_y_true, final_predictions_f1_weighted)

print("\nOverall Metrics (Weighted Average F1 Ensemble):")
print(f"Overall Accuracy: {overall_accuracy}")
print(f"Overall Precision: {overall_precision}")
print(f"Overall Recall: {overall_recall}")
print(f"Overall F1 Score: {overall_f1}")

##########################
# Weighted Average Cluster Size Ensemble
##########################
overall_accuracy = accuracy_score(all_y_true, final_predictions_cluster_weighted)
overall_precision = precision_score(all_y_true, final_predictions_cluster_weighted)
overall_recall = recall_score(all_y_true, final_predictions_cluster_weighted)
overall_f1 = f1_score(all_y_true, final_predictions_cluster_weighted)

print("\nOverall Metrics (Weighted Average Cluster Size Ensemble):")
print(f"Overall Accuracy: {overall_accuracy}")
print(f"Overall Precision: {overall_precision}")
print(f"Overall Recall: {overall_recall}")
print(f"Overall F1 Score: {overall_f1}")

##########################
# Weighted Average Probability Ensemble
##########################
overall_accuracy = accuracy_score(all_y_true, final_predictions_probability_weighted)
overall_precision = precision_score(all_y_true, final_predictions_probability_weighted)
overall_recall = recall_score(all_y_true, final_predictions_probability_weighted)
overall_f1 = f1_score(all_y_true, final_predictions_probability_weighted)

print("\nOverall Metrics (Weighted Average Probability Ensemble):")
print(f"Overall Accuracy: {overall_accuracy}")
print(f"Overall Precision: {overall_precision}")
print(f"Overall Recall: {overall_recall}")
print(f"Overall F1 Score: {overall_f1}")

##########################
# Non-Weighted Ensemble
##########################
overall_accuracy = accuracy_score(all_y_true, final_predictions_non_weighted)
overall_precision = precision_score(all_y_true, final_predictions_non_weighted)
overall_recall = recall_score(all_y_true, final_predictions_non_weighted)
overall_f1 = f1_score(all_y_true, final_predictions_non_weighted)

print("\nOverall Metrics (Non-Weighted):")
print(f"Overall Accuracy: {overall_accuracy}")
print(f"Overall Precision: {overall_precision}")
print(f"Overall Recall: {overall_recall}")
print(f"Overall F1 Score: {overall_f1}")

##########################
# Pre-Classifier
##########################
overall_accuracy = accuracy_score(all_y_true, final_predictions_pre_classifier)
overall_precision = precision_score(all_y_true, final_predictions_pre_classifier)
overall_recall = recall_score(all_y_true, final_predictions_pre_classifier)
overall_f1 = f1_score(all_y_true, final_predictions_pre_classifier)

print("\nOverall Metrics (Pre-Classifier):")
print(f"Overall Accuracy: {overall_accuracy}")
print(f"Overall Precision: {overall_precision}")
print(f"Overall Recall: {overall_recall}")
print(f"Overall F1 Score: {overall_f1}")

##########################
# Entropy Weighted
##########################
overall_accuracy = accuracy_score(all_y_true, final_predictions_entropy_weighted)
overall_precision = precision_score(all_y_true, final_predictions_entropy_weighted)
overall_recall = recall_score(all_y_true, final_predictions_entropy_weighted)
overall_f1 = f1_score(all_y_true, final_predictions_entropy_weighted)

print("\nOverall Metrics (Entropy Weighted):")
print(f"Overall Accuracy: {overall_accuracy}")
print(f"Overall Precision: {overall_precision}")
print(f"Overall Recall: {overall_recall}")
print(f"Overall F1 Score: {overall_f1}")


Overall Metrics (Weighted Average F1 Ensemble):
Overall Accuracy: 0.6793669094911168
Overall Precision: 0.34872448979591836
Overall Recall: 0.27716950527169504
Overall F1 Score: 0.3088567555354722

Overall Metrics (Weighted Average Cluster Size Ensemble):
Overall Accuracy: 0.615481368901001
Overall Precision: 0.3151421983089931
Overall Recall: 0.41565287915652877
Overall F1 Score: 0.3584856168575675

Overall Metrics (Weighted Average Probability Ensemble):
Overall Accuracy: 0.6339290393585242
Overall Precision: 0.32286453839516827
Overall Recall: 0.3793592862935929
Overall F1 Score: 0.3488393772723035

Overall Metrics (Non-Weighted):
Overall Accuracy: 0.6123892877731775
Overall Precision: 0.3181818181818182
Overall Recall: 0.43714517437145173
Overall F1 Score: 0.3682951827810044

Overall Metrics (Pre-Classifier):
Overall Accuracy: 0.6188878989570777
Overall Precision: 0.3220802919708029
Overall Recall: 0.42944038929440387
Overall F1 Score: 0.3680917622523462

Overall Metrics (Entropy 

In [102]:
# final_predictions_entropy_weighted = []
# all_y_true = []
# for idx in X_test.index:
    
#     votes_entropy_weighted = {}
#     for model_cluster, model in rf_models.items():
#         prediction = model.predict(X_test.drop(columns=['cluster']).loc[[idx]])[0]
        
#         if prediction in votes_entropy_weighted:
#             votes_entropy_weighted[prediction] += entropy_weights[model_cluster]
#         else:
#             votes_entropy_weighted[prediction] = entropy_weights[model_cluster]    

#     # Final prediction is the weighted average
#     final_prediction_entropy_weighted = max(votes_entropy_weighted, key=votes_entropy_weighted.get)
#     final_predictions_entropy_weighted.append(final_prediction_entropy_weighted)
    
#     all_y_true.append(y_test.loc[idx])
    
# overall_accuracy = accuracy_score(all_y_true, final_predictions_entropy_weighted)
# overall_precision = precision_score(all_y_true, final_predictions_entropy_weighted)
# overall_recall = recall_score(all_y_true, final_predictions_entropy_weighted)
# overall_f1 = f1_score(all_y_true, final_predictions_entropy_weighted)

# print("Overall Metrics (Entropy Weighted):")
# print(f"Overall Accuracy: {overall_accuracy}")
# print(f"Overall Precision: {overall_precision}")
# print(f"Overall Recall: {overall_recall}")
# print(f"Overall F1 Score: {overall_f1}")

In [52]:
# # Make final predictions on the test set using weighted average
# final_predictions_pre_classifier = []
# all_y_true = []
# for idx in X_test.index:
        
    
    
#     votes_pre_classifier = {}
#     for model_cluster, model in rf_models.items():
#         prediction = model.predict(X_test.drop(columns=['cluster']).loc[[idx]])[0]
        
            

#     # Final prediction is the weighted average
#     final_prediction_pre_classifier = max(votes_pre_classifier, key=votes_pre_classifier.get)
#     final_predictions_pre_classifier.append(final_prediction_pre_classifier)
    
#     all_y_true.append(y_test.loc[idx])

In [53]:
# overall_accuracy = accuracy_score(all_y_true, final_predictions_pre_classifier)
# overall_precision = precision_score(all_y_true, final_predictions_pre_classifier)
# overall_recall = recall_score(all_y_true, final_predictions_pre_classifier)
# overall_f1 = f1_score(all_y_true, final_predictions_pre_classifier)

# print("\nOverall Metrics (Pre-Classifier):")
# print(f"Overall Accuracy: {overall_accuracy}")
# print(f"Overall Precision: {overall_precision}")
# print(f"Overall Recall: {overall_recall}")
# print(f"Overall F1 Score: {overall_f1}")


Overall Metrics (Pre-Classifier):
Overall Accuracy: 0.6125465122373042
Overall Precision: 0.3161512027491409
Overall Recall: 0.42903487429034876
Overall F1 Score: 0.3640430107526882


In [54]:
# entropy_weights = {}
# for model_cluster, model in rf_models.items():
#     # Calculate the relative error of the sub-network
#     X_cluster = X_train[X_train['cluster'] == model_cluster].drop(columns=['cluster'])
#     y_cluster = y_train.loc[X_cluster.index]
#     predictions = model.predict(X_cluster)
#     relative_errors = (predictions != y_cluster).astype(int)
#     p_error = relative_errors.sum() / len(relative_errors)
        
#     # Calculate the entropy based on relative error
#     entropy = -p_error * np.log(p_error + 1e-9) - (1 - p_error) * np.log(1 - p_error + 1e-9)
#     weight = 1 - entropy
#     entropy_weights[model_cluster] = weight
    
# # Normalize weights


In [ ]:
# 

In [60]:
# total_weight = sum(entropy_weights.values())
# for model_cluster in entropy_weights:
#     entropy_weights[model_cluster] /= total_weight
# entropy_weights

{0: 0.10432004801356558,
 1: 0.10134829212649679,
 2: 0.09514352808084782,
 3: 0.09633512519255015,
 4: 0.10003645121486365,
 5: 0.09598469948262066,
 6: 0.1011535880558346,
 7: 0.102361315957685,
 8: 0.10035321546928196,
 9: 0.10296373640625374}